In [ ]:
!pip3 install konlpy

In [ ]:
from konlpy.tag import Kkma
tokenizer = Kkma()

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
training_data = pd.read_csv("training.csv")
training_data['X'] = training_data['X'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ",regex=True)
training_data['X'] = training_data['X'].str.replace('^ +', "",regex=True) # 공백은 empty 값으로 변경
training_data['X'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
training_data = training_data.dropna(how='any') # Null 값 제거

validation_data = pd.read_csv("validation.csv")
validation_data['X'] = validation_data['X'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ",regex=True)
validation_data['X'] = validation_data['X'].str.replace('^ +', "",regex=True) # 공백은 empty 값으로 변경
validation_data['X'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
validation_data = validation_data.dropna(how='any') # Null 값 제거

test_data = pd.read_csv("test.csv")
test_data['X'] = test_data['X'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ",regex=True)
test_data['X'] = test_data['X'].str.replace('^ +', "",regex=True) # 공백은 empty 값으로 변경
test_data['X'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

practice_data = pd.read_csv("practice.csv")
practice_data['X'] = practice_data['X'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ",regex=True)
practice_data['X'] = practice_data['X'].str.replace('^ +', "",regex=True) # 공백은 empty 값으로 변경
practice_data['X'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
practice_data = practice_data.dropna(how='any') # Null 값 제거

In [ ]:
Lyric_train = training_data['X'].values
Lyric_validation = validation_data['X'].values
Lyric_test = test_data['X'].values
Lyric_practice = practice_data['X'].values
y_train_temp = training_data['y'].values
y_validation_temp = validation_data['y'].values
y_test_temp = test_data['y'].values
y_practice_temp = practice_data['y'].values

print("# split done")

In [ ]:
X_train_temp = []
X_validation_temp = []
X_test_temp = []
X_practice_temp = []

for i in Lyric_train:
    token = tokenizer.morphs(i)
    X_train_temp.append(token)
for i in Lyric_validation:
    token = tokenizer.morphs(i)
    X_validation_temp.append(token)
for i in Lyric_test:
    token = tokenizer.morphs(i)
    X_test_temp.append(token)
for i in Lyric_practice:
    token = tokenizer.morphs(i)
    X_practice_temp.append(token)

print("# tokenization done")

In [ ]:
from konlpy.tag import Kkma
tokenizer = Kkma()
X_practice_temp = []
Lyric_practice = practice_data['X'].values
for i in Lyric_practice:
    token = tokenizer.morphs(i)
    X_practice_temp.append(token)

In [ ]:
# One-Hot encoding 결과 구현

y_train = pd.get_dummies(y_train_temp).values
y_validation = pd.get_dummies(y_validation_temp).values
y_test = pd.get_dummies(y_test_temp).values
y_practice = pd.get_dummies(y_practice_temp).values

In [ ]:
#New Learning

X_train = X_train_temp
X_validation = X_validation_temp
X_test = X_test_temp
X_practice = X_practice_temp

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
count = 0
for word, word_count in tokenizer.word_counts.items():
    if word_count > 1:
        count += 1
print(count)

In [ ]:
vocab_size = 11335
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = tokenizer.texts_to_sequences(X_train)
X_validation = tokenizer.texts_to_sequences(X_validation)
X_test = tokenizer.texts_to_sequences(X_test)
X_practice = tokenizer.texts_to_sequences(X_practice)

print("# int_encoding done")
max_len = 100
X_train = pad_sequences(X_train, maxlen=max_len)
X_validation = pad_sequences(X_validation, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_practice = pad_sequences(X_practice, maxlen=max_len)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Embedding, Dropout

embedding_dim = 100
hidden_units = 128
num_classes = 4

# 모델 구축
# 레이어들을 쌓을 모델을 생성
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 테스트 데이터 손실함수값(val_acc)이 patience회 이상 연속 증가하면 학습을 조기 종료하는 콜백
early_stop = EarlyStopping(monitor='val_acc', mode='min', verbose=1, patience=10)
# 훈련 도중 테스트 데이터 정확도(val_acc)가 높았던 순간을 체크포인트로 저장해 활용하는 콜백
model_check = ModelCheckpoint('the_best.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
hist = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), epochs=50, batch_size=16, callbacks=[early_stop, model_check])

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(hist.history['acc']) + 1)
plt.plot(epochs, hist.history['loss'])
plt.plot(epochs, hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(epochs, hist.history['acc'])
plt.plot(epochs, hist.history['val_acc'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# 정확도 측정
# 출력하면 [loss, acc]
from tensorflow.keras.models import load_model

loaded_model = load_model('the_best.h5')
print("테스트 정확도 : %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
# confusion matrix 사용을 위한 라이브러리
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import itertools

# confusion matrix 그리는 함수 
def plot_confusion_matrix(con_mat, labels, title='Confusion Matrix', cmap=plt.cm.get_cmap('Blues'), normalize=False):
    plt.imshow(con_mat, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    marks = np.arange(len(labels))
    nlabels = []
    for k in range(len(con_mat)):
        n = sum(con_mat[k])
        nlabel = '{0}(n={1})'.format(labels[k],n)
        nlabels.append(nlabel)
    plt.xticks(marks, labels)
    plt.yticks(marks, nlabels)

    thresh = con_mat.max() / 2.
    if normalize:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, '{0}'.format(con_mat[i, j]), horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    else:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, con_mat[i, j], horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

predictions = loaded_model.predict(X_test)
labels = ['sad', 'fear', 'happiness', 'anger']
# 예측값과 참값 
pred_labels = np.argmax(predictions, axis=1)
y_ = []
for i in y_test:
    if i[0] == 1:
        y_.append(0)
    elif i[1] == 1:
        y_.append(1)
    elif i[2] == 1:
        y_.append(2)
    else:
        y_.append(3)
true_labels = y_
#메인 실행 
confusion_matrix = confusion_matrix(true_labels, pred_labels)
plot_confusion_matrix(confusion_matrix, labels=labels, normalize=True)

In [ ]:
print(confusion_matrix)

In [ ]:
#precision

print("precision_score : ", precision_score(true_labels, pred_labels, average="weighted"))

#recall

print("recall_score : ", recall_score(true_labels, pred_labels, average="weighted"))

#f1score

print("f1_score : ", f1_score(true_labels, pred_labels, average="weighted"))

Multi Demo

In [ ]:
y_pred = model.predict(X_practice)
result = []
for i in range(len(y_pred)):
    if np.argmax(y_pred[i]) == 0:
        result.append("슬픔")
    elif np.argmax(y_pred[i]) == 1:
        result.append("두렵")
    elif np.argmax(y_pred[i]) == 2:
        result.append("행복")
    else:
        result.append("분노")

for i in range(len(practice_data)):
    print("제목 : ", practice_data['제목'][i], ", 기존 감정 : ", practice_data['y'][i], ", 예측 감정 : ", result[i])

Binary

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Embedding, Dropout

embedding_dim = 100
hidden_units = 128
num_classes = 2

# 모델 구축
# 레이어들을 쌓을 모델을 생성
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='sigmoid'))

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 테스트 데이터 손실함수값(val_acc)이 patience회 이상 연속 증가하면 학습을 조기 종료하는 콜백
early_stop = EarlyStopping(monitor='val_acc', mode='min', verbose=1, patience=15)
# 훈련 도중 테스트 데이터 정확도(val_acc)가 높았던 순간을 체크포인트로 저장해 활용하는 콜백
model_check = ModelCheckpoint('the_best2.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
hist = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), epochs=50, batch_size=16, callbacks=[early_stop, model_check])

In [ ]:
# 정확도 측정
# 출력하면 [loss, acc]
from tensorflow.keras.models import load_model

loaded_model = load_model('the_best2.h5')
print("테스트 정확도 : %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(hist.history['acc']) + 1)
plt.plot(epochs, hist.history['loss'])
plt.plot(epochs, hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(epochs, hist.history['acc'])
plt.plot(epochs, hist.history['val_acc'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# confusion matrix 사용을 위한 라이브러리
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import itertools

# confusion matrix 그리는 함수 
def plot_confusion_matrix(con_mat, labels, title='Confusion Matrix', cmap=plt.cm.get_cmap('Blues'), normalize=False):
    plt.imshow(con_mat, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    marks = np.arange(len(labels))
    nlabels = []
    for k in range(len(con_mat)):
        n = sum(con_mat[k])
        nlabel = '{0}(n={1})'.format(labels[k],n)
        nlabels.append(nlabel)
    plt.xticks(marks, labels)
    plt.yticks(marks, nlabels)

    thresh = con_mat.max() / 2.
    if normalize:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, '{0}'.format(con_mat[i, j]), horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    else:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, con_mat[i, j], horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

predictions = loaded_model.predict(X_test)
labels = ['minus', 'plus']
# 예측값과 참값 
pred_labels = np.argmax(predictions, axis=1)
y_ = []
for i in y_test:
    if i[0] == 1:
        y_.append(0)
    else:
        y_.append(1)
true_labels = y_
#메인 실행 
confusion_matrix = confusion_matrix(true_labels, pred_labels)
plot_confusion_matrix(confusion_matrix, labels=labels, normalize=True)

In [ ]:
#precision

print("precision_score : ", precision_score(true_labels, pred_labels, average="weighted"))

#recall

print("recall_score : ", recall_score(true_labels, pred_labels, average="weighted"))

#f1scord

print("f1_score : ", f1_score(true_labels, pred_labels, average="weighted"))

Binary Demo

In [ ]:
y_pred = model.predict(X_practice)
result = []
for i in range(len(y_pred)):
    if np.argmax(y_pred[i]) == 0:
        result.append("부정")
    else:
        result.append("긍정")

for i in range(len(practice_data)):
    print("제목 : ", practice_data['제목'][i], ", 기존 감정 : ", practice_data['y'][i], ", 예측 감정 : ", result[i])